# Control Logic for DiP Systolic Array


In [1]:
from hardware_accelerators import BF16, float_adder, lmul_fast
from hardware_accelerators.rtllib import AcceleratorConfig
from hardware_accelerators.simulation import (
    SystolicArraySimulator,
    MatrixEngineSimulator,
)
import numpy as np

In [2]:
sim = SystolicArraySimulator(3, BF16, BF16, lmul_fast, float_adder, True)

weights = np.identity(3)
activations = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

result = sim.simulate(weights, activations)
sim._step()
sim._step()
result

array([[1.0625, 2.125 , 3.125 ],
       [4.25  , 5.25  , 6.25  ],
       [7.25  , 8.5   , 9.5   ]])

In [3]:
sim.history

[
 Simulation State - Step 0
 ----------------------------------------
 Inputs:
   w_en: 1
   enable: 0
   weights: [0. 0. 0.]
   data: [0. 0. 0.]
 
 Weights Matrix:
 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]
 
 Data Matrix:
 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]
 
 Accumulators:
 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]
 
 Control Registers:
 {'data_controls': [0, [0, 0]], 'accum_controls': [0, 0, 0], 'control_out': 0, 'mul_controls': [0, 0, 0]}
 
 Outputs:
 [0. 0. 0.]
 ----------------------------------------,
 
 Simulation State - Step 1
 ----------------------------------------
 Inputs:
   w_en: 1
   enable: 0
   weights: [0. 0. 0.]
   data: [0. 0. 0.]
 
 Weights Matrix:
 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]
 
 Data Matrix:
 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]
 
 Accumulators:
 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]
 
 Control Registers:
 {'data_controls': [0, [0, 0]], 'accum_controls': [0, 0, 0], 'control_out': 0, 'mul_controls': [0, 0, 0]}
 
 Outputs:
 [0. 0. 0.]
 --------------

# Connecting everything together


In [4]:
config = AcceleratorConfig(
    array_size=3,
    data_type=BF16,
    weight_type=BF16,
    accum_type=BF16,
    pe_adder=float_adder,
    pe_multiplier=lmul_fast,
    accum_adder=float_adder,
    pipeline=False,
    accumulator_tiles=4,
)


sim = MatrixEngineSimulator(config)

# Load data
print(f"Original Weights:\n{weights}")
print(f"Original Activations:\n{activations}\n")

sim.load_weights(weights, bank=0)
sim.load_activations(activations, bank=0)

print(f"Weights:\n{sim.weight_banks}")
print(f"Activations:\n{sim.data_banks}")

Original Weights:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
Original Activations:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

Weights:
[[[1. 1. 1.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
Activations:
[[[1. 2. 3.]
  [4. 5. 6.]
  [7. 8. 9.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [13]:
sim = MatrixEngineSimulator(config)

# Load data
print(f"Original Weights:\n{weights}")
print(f"Original Activations:\n{activations}\n")

sim.load_weights(weights, bank=0)
sim.load_activations(activations, bank=0)

# Perform computation
sim.matmul(data_bank=0, weight_bank=0, accum_tile=0, accumulate=False)

# Debug
sim.print_history(inputs=True, memory_buffer=False, accumulator=True)

Original Weights:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
Original Activations:
[[1 2 3]
 [4 5 6]
 [7 8 9]]


Simulation Step 0

Input Signals:
--------------------------------------------------------------------------------
  data_start: 0
  data_bank: 0
  weight_start: 1
  weight_bank: 0
  accum_start: 0
  accum_tile_addr: 0
  accum_mode: 0
  accum_read_start: 0
  accum_read_tile_addr: 0

Systolic Array State:
--------------------------------------------------------------------------------

Inputs:
  w_en: 0
  enable: 0
  weights: [0. 0. 0.]
  data: [0. 0. 0.]

Weights Matrix:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

Data Matrix:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

Accumulators:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

Control Registers:
{'data_controls': [0, [0, 0]], 'accum_controls': [0, 0, 0], 'control_out': 0}

Outputs:
[0. 0. 0.]
----------------------------------------


Accumulator Memory State:
--------------------------------------------------------------------------------
Tile Sta

In [15]:
accum_tile_0 = sim.read_accumulator_tile(0)
sim.step()
accum_tile_0

array([[7.25  , 8.5   , 9.5   ],
       [4.25  , 5.25  , 6.25  ],
       [1.0625, 2.125 , 3.125 ]])

In [16]:
sim.print_history(memory_buffer=False, systolic_array=False)


Simulation Step 0

Input Signals:
--------------------------------------------------------------------------------
  data_start: 0
  data_bank: 0
  weight_start: 1
  weight_bank: 0
  accum_start: 0
  accum_tile_addr: 0
  accum_mode: 0
  accum_read_start: 0
  accum_read_tile_addr: 0

Accumulator Memory State:
--------------------------------------------------------------------------------
Tile States:
Tile 0:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Tile 1:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Tile 2:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Tile 3:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

Accumulator Output Ports:
[0. 0. 0.]




Simulation Step 1

Input Signals:
--------------------------------------------------------------------------------
  data_start: 0
  data_bank: 0
  weight_start: 0
  weight_bank: 0
  accum_start: 0
  accum_tile_addr: 0
  accum_mode: 0
  accum_read_start: 0
  accum_read_tile_addr: 0

Accumulator Memory State:
---------------------------------------------------------------